# Modelos RNN de sequência para sequência: tarefa de tradução

Neste projeto vamos explorar os fundamentos dos modelos de sequência para sequência e implementar um modelo baseado em RNN para uma tarefa de tradução usando o PyTorch.

### Objetivos
- Compreender a arquitetura de redes neurais recorrentes (RNN)
- Criar um modelo codificador-decodificador para uma tarefa de tradução
- Treinar e avaliar o modelo
- Criar um gerador para a tarefa de tradução
- Conceitos relacionados à Perplexidade e à pontuação BLEU e usá-los para avaliar traduções

### Preparar setup - instalar bibliotecas

In [1]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [2]:
check_and_install('torchtext')
check_and_install('torch')
check_and_install('spacy')
check_and_install('torchdata')
check_and_install('portalocker')
check_and_install('nltk')

torchtext já está instalado.
torch já está instalado.
spacy já está instalado.
torchdata já está instalado.
portalocker já está instalado.
nltk já está instalado.


In [3]:
#!python -m spacy download en_core_web_sm
#!python -m spacy download de_core_news_sm

### Importar bibliotecas

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext
from torchtext.vocab import build_vocab_from_iterator
from nltk.translate.bleu_score import sentence_bleu
import torch
import torch.nn as nn
import torch.optim as optim


import numpy as np
import random
import math
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

### Background

Os modelos de sequência para sequência (Seq2seq) revolucionaram diversas tarefas de processamento de linguagem natural (NLP), como tradução automática, sumarização de texto e chatbots. Esses modelos utilizam redes neurais recorrentes (RNNs) para processar sequências de entrada de comprimento variável e gerar sequências de saída também de comprimento variável.
#### História dos modelos de sequência para sequência
Os modelos seq2seq foram introduzidos como uma extensão das redes neurais feedforward tradicionais. Pesquisadores perceberam a necessidade de modelos que pudessem lidar com sequências de entrada e saída de comprimentos variáveis, como ocorre na tradução automática. O trabalho pioneiro de Sutskever et al. (2014) introduziu o uso de RNNs para modelos seq2seq.

Principais objetivos dos modelos seq2seq:
- Tradução: Traduzir uma sequência de um domínio para outro (e.g., inglês para francês).
- Resposta a perguntas: Gerar uma resposta em linguagem natural com base em uma frase de entrada (e.g., chatbots).
- Sumarização: Resumir um documento longo em uma sequência mais curta de frases.
- Outras aplicações: Qualquer tarefa que envolva geração de sequência, como geração de legendas para imagens, descrições automáticas e muito mais.







### Introdução as RNNs
RNNs são uma classe de redes neurais projetadas para processar dados sequenciais.
Elas mantêm uma memória interna ($h_t$) para capturar informações de etapas anteriores e usá-las para previsões atuais.
RNNs têm uma conexão recorrente que permite que as informações fluam de uma etapa para a próxima.
Redes Neurais Recorrentes (RNNs) operam em sequências e utilizam estados anteriores para influenciar o estado atual. Aqui está a formulação geral de uma RNN simples:

Dado:

-$ \mathbf{x}_t $: vetor de entrada no passo de tempo $t$

-$ \mathbf{h}_{t-1} $: vetor de estado oculto do passo de tempo anterior

-$ \mathbf{W}_x $ e $ \mathbf{W}_h $: matrizes de peso para o estado de entrada e oculto, respectivamente

-$ \mathbf{b} $: vetor de polarização

-$\sigma$: função de ativação (geralmente uma sigmoide ou tanh)

As equações de atualização para o estado oculto $ \mathbf{h}_t $ e a saída $ \mathbf{y}_t $ são as seguintes:

$$
\begin{align*}
\mathbf{h}_t &= \sigma(\mathbf{W}_x \cdot \mathbf{x}_t + \mathbf{W}_h \cdot \mathbf{h}_{t-1} + \mathbf{b})
\end{align*}
$$

Pode ser visto que a função de estado oculto depende do estado oculto anterior, bem como da entrada no tempo t, razão pela qual ela tem uma memória coletiva de passos de tempo anteriores.

Para a saída (se você estiver fazendo uma previsão em cada passo de tempo):
$$
\begin{align*}
\mathbf{y}_t &= \text{softmax}(\mathbf{W}_o \cdot \mathbf{h}_t + \mathbf{b}_o)
\end{align*}
$$
Onde:

$ \mathbf{W}_o $: matriz de peso para a saída E $ \mathbf{b}_o$: vetor de viés para a saída

Dependendo da tarefa específica, uma célula RNN pode produzir uma saída de $h_t$ ou transferi-la somente para a célula seguinte, servindo como memória interna. Embora a capacidade da arquitetura de reter memória possa parecer ilusória à primeira vista, vamos elucidar isso implementando uma RNN simples para manipular o seguinte mecanismo de dados:
![a title](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Screenshot%202023-10-19%20at%2011.29.23%E2%80%AFAM.png)
O diagrama mostra uma máquina de estados ou modelo de transição com três estados distintos, representados pelos círculos roxos proeminentes. Cada estado é distintamente rotulado com um valor para $ h $: $ h = -1 $, $ h = 0 $ e $ h = 1 $.

1. **Estado $ h = -1 $**:
- Mantém-se quando $ x = 1 $ (ilustrado pelo loop amarelo).
- Prossegue para o estado $ h = 0 $ ao receber $ x = -1 $ (destacado pela seta vermelha).

2. **Estado $ h = 0 $**:
- Move-se para o estado $ h = -1 $ quando $ x = 1 $ (ilustrado pela seta vermelha).
- Avança para o estado $ h = 1 $ com $ x = -1 $ (marcado pela seta vermelha).

3. **Estado $ h = 1 $**:
- Mantém sua posição quando $ x = -1 $ (indicado pelo loop amarelo).
- Transita para o estado $ h = 0 $ ao receber $ x = 1 $ (representado pela seta vermelha).

Para encapsular, o diagrama retrata efetivamente transições entre três estados com base na entrada $ x $. Contingente no estado predominante e na entrada $ x $, a máquina de estados transita para um estado diferente ou permanece estacionária.

Podemos representar a máquina de estados mencionada anteriormente usando a camada detalhada abaixo. Use $tanh$, pois o valor $h$ deve cair entre [-1, 1]. Note que você excluiu a saída para simplificação:
$$\begin{align*}
W_{xh} & = -10.0 \\
W_{hh} & = 10.0 \\
b_h & = 0.0 \\
x_t & = 1 \\
h_{\text{prev}} & = 0.0 \\
h_t & = \tanh(x_t \cdot W_{xh} + h_{\text{prev}} \cdot W_{hh} + b_h)
\end{align*}$$

In [5]:
W_xh = torch.tensor(-10.0)
W_hh = torch.tensor(10.0)
b_h = torch.tensor(0.0)
x_t = 1
h_prev = torch.tensor(-1)

Considerando a seguinte sequência $x_t$ para $t=0,1,..,7$,

In [6]:
X = [1, 1, -1, -1, 1, 1]

Supondo que começamos no estado inicial $h = 0$, com o vetor de entrada $x$ acima, o vetor de estado $h$ deve ficar assim:

In [7]:
H=[-1,-1,0,1,0,-1]

In [8]:
# Initialize an empty list to store the predicted state values
H_hat = []

# Loop through each data point in the input sequence X
t = 1
for x in X:
    # Assign the current data point to x_t
    print("t = ", t)
    x_t = x

    # Print the value of the previous state (h at time t-1)
    print("h_t-1", h_prev.item())

    # Compute the current state (h at time t) using the RNN formula with tanh activation
    h_t = torch.tanh(x_t * W_xh + h_prev * W_hh + b_h)

    # Update h_prev to the current state value for the next iteration
    h_prev = h_t

    # Print the current input value (x at time t)
    print("x_t", x_t)

    # Print the computed state value (h at time t)
    print("h_t", h_t.item())
    
    # Append the current state value to the H_hat list after converting it to integer
    H_hat.append(int(h_t.item()))
    t += 1
    print("---")

t =  1
h_t-1 -1
x_t 1
h_t -1.0
---
t =  2
h_t-1 -1.0
x_t 1
h_t -1.0
---
t =  3
h_t-1 -1.0
x_t -1
h_t 0.0
---
t =  4
h_t-1 0.0
x_t -1
h_t 1.0
---
t =  5
h_t-1 1.0
x_t 1
h_t 0.0
---
t =  6
h_t-1 0.0
x_t 1
h_t -1.0
---


Você pode avaliar a precisão do estado previsto ```H_hat``` comparando-o ao estado real ```H```. Em RNNs, o estado $ h_t $ é utilizado para prever uma sequência de saída $y_t $ com base na sequência de entrada fornecida $ x_t $.

In [9]:
H_hat

[-1, -1, 0, 1, 0, -1]

In [10]:
H

[-1, -1, 0, 1, 0, -1]

Embora tenhamos predefinido $W_{xh}$, $W_{hh}$ e $b_h$, na prática esses valores precisam ser identificados por meio de treinamento em dados.

Na prática, modificações e melhorias, como Long Short-Term Memory (LSTM) e Gated Recurrent Units (GRU), são frequentemente usadas para resolver problemas como o problema do gradiente de desaparecimento em RNNs básicas.

Uma célula LSTM tem três componentes principais: uma porta de entrada (input gate), uma porta de esquecimento (forget gate) e uma porta de saída (output gate).
- A **porta de entrada** controla quanta informação nova deve ser armazenada na memória da célula. Ela olha para a entrada atual e o estado oculto anterior e decide quais partes da nova entrada lembrar.
- A **porta de esquecimento** determina quais informações devem ser descartadas ou esquecidas da memória da célula. Ela considera a entrada atual e o estado oculto anterior e decide quais partes da memória anterior não são mais relevantes.
- A **porta de saída** determina quais informações devem ser emitidas da célula. Ela olha para a entrada atual e o estado oculto anterior e decide quais partes da memória da célula incluir na saída.

A ideia-chave por trás das células LSTM é que elas têm um estado de memória separado que pode reter ou esquecer informações seletivamente ao longo do tempo. Isso as ajuda a lidar com dependências de longo alcance e lembrar informações importantes de etapas anteriores em uma sequência.

### Arquitetura de sequência para sequência (sequence-to-sequence)

Os modelos Seq2seq têm uma estrutura Encoder-Decoder. O codificador codifica a sequência de entrada em uma representação de dimensão fixa, geralmente chamada de vetor de contexto($h_t$). O decodificador gera a sequência de saída com base no vetor de contexto codificado.

Vamos olhar mais de perto as caixas codificadora e decodificadora no vídeo abaixo. A tradução é uma tarefa típica de sequência para sequência. A entrada é uma sequência de palavras no idioma original ("Eu amo viajar"), enquanto a saída é sua tradução no idioma de destino ("J'adore voyager"). Conforme mostrado no vídeo, a entrada é alimentada na parte decodificadora, uma palavra após a outra. Cada célula RNN recebe uma palavra ($x_t$) e tem uma memória interna ($h_t$). Após processar a entrada e $h_t$, a célula RNN passa um vetor de contexto atualizado ($h_{t+1}$) para a próxima célula RNN. Quando o fim da frase é alcançado, o vetor de contexto é passado para a parte decodificadora. As células decodificadoras também são células RNN que recebem o vetor de contexto e geram a saída palavra por palavra. Cada RNN recebe a palavra gerada, bem como o vetor de contexto atualizado de sua célula anterior e gera a próxima palavra ($y_t$). Essa arquitetura permite gerar texto sem restrições de comprimento.

<video width="640" height="480"
src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Translation_RNN.mp4"
controls>
</video>


## Implementação do codificador (Encoder) no PyTorch

Para implementar a parte do codificador (encoder) usando o Pytorch, vamos criar a subclasse da classe torch.nn.Module e definirá os métodos __init__() e __forward__().

Primeiro definimos os parâmetros que são usados na função __init__():
- O `vocab_len` nada mais é do que o número de palavras únicas presentes no vocabulário. Após o pré-processamento dos dados, você pode contar o número de palavras únicas no seu vocabulário e usar essa contagem aqui. Esta será a dimensão da entrada do modelo.
- O embedding_dim é a dimensão de saída do vetor de incorporação que você precisa. Uma boa prática é usar 256-512 para um aplicativo de demonstração de exemplo como o que você está construindo aqui.
- O LSTM pode de fato ser empilhado, permitindo múltiplas camadas. Na implementação inicial, você usará apenas uma camada. No entanto, para acomodar a flexibilidade futura, você passará o parâmetro `n_layers` para especificar o número de camadas no LSTM.
- `hid_dim` é a dimensionalidade dos estados oculto e de célula.
- `dropout` é a quantidade de dropout a ser usada. Este é um parâmetro de regularização para evitar overfitting.

Camadas:
- A camada Embedding pega os dados de entrada e gera o vetor de embedding, portanto, a dimensão deles precisa ser definida como `vocab_len` e `embedding_dim`.
- A camada LSTM pega o `embedding_dim` como os dados de entrada e cria um total de 3 saídas: `hidden`, `cell` e `output`. Aqui você precisa definir o número de neurônios que precisa no LSTM, que é definido usando o `hid_dim`.

Na função __forward__(), a camada Embedding é definida, que utiliza o `vocab_len` para converter internamente o input_batch em uma representação one-hot. Em seguida, a camada LSTM recebe a entrada incorporada e produz três vetores: Output, Hidden e cell. Quanto ao codificador, você não precisa do vetor de saída do LSTM, pois você passa apenas o contexto vector(`hidden`+`cell`) para o bloco decoder. Portanto, forward() retorna apenas hidden e cell.

Nota: Ao usar um LSTM, você tem um estado de célula adicional. No entanto, se você estivesse usando uma GRU, você teria apenas o estado hidden.

In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, droput_prob):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_len, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = droput_prob)
        self.dropout = nn.Dropout(droput_prob)

    def forward(self, input_batch):
        # input_batch = [src len, batch size]
        embed = self.dropout(self.embedding(input_batch))
        embed = embed.to(device)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        outputs, (hidden, cell) = self.lstm(embed)

        return hidden, cell

In [12]:
# Create an encoder instance
vocab_len = 8
emb_dim = 10
hid_dim=8
n_layers=1
dropout_prob=0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder_t = Encoder(vocab_len, emb_dim, hid_dim, n_layers, dropout_prob).to(device)

Testar com um exemplo simples onde o método de encaminhamento do codificador transforma a frase `src` em estados `hidden` e `cell`. tensor([[0],[3],[4],[2],[1]]) é igual a `src` = 0,3,4,2,1 em que cada número representa um token no vocabulário `src`. Por exemplo, 0:`<bos>`,3:"Das", 4:"ist",2:"schön", 1:`<eos>`. Observe que aqui temos um tamanho de lote de 1.

In [13]:
src_batch = torch.tensor([[0,3,4,2,1]])

In [14]:
# you need to transpose the input tensor as the encoder LSTM is in Sequence_first mode by default
src_batch = src_batch.t().to(device)
print("Shape of input(src) tensor:", src_batch.shape)

Shape of input(src) tensor: torch.Size([5, 1])


In [15]:
hidden_t , cell_t = encoder_t(src_batch)
print("Hidden tensor from encoder:",hidden_t ,"\nCell tensor from encoder:", cell_t)

Hidden tensor from encoder: tensor([[[ 0.0073,  0.2685, -0.0821, -0.1150,  0.1230, -0.0823, -0.1276,
           0.0178]]], grad_fn=<StackBackward0>) 
Cell tensor from encoder: tensor([[[ 0.0381,  0.3959, -0.3196, -0.1924,  0.1916, -0.1427, -0.2526,
           0.0671]]], grad_fn=<StackBackward0>)


O codificador pega toda a sequência de origem como entrada, que consiste em uma sequência de palavras ou tokens. O codificador LSTM processa toda a sequência de entrada e atualiza seus estados ocultos a cada passo de tempo. Os estados ocultos da rede LSTM agem como uma forma de memória e capturam as informações contextuais da sequência de entrada. Após processar toda a sequência de entrada, o estado oculto final do codificador LSTM captura a representação resumida do contexto da sequência de entrada. Este estado oculto final é algumas vezes chamado de "vetor de contexto".

### Implementação do decodificador (Decoder) no PyTorch
Para entender melhor o mecanismo interno da parte do decodificador, vamos analisar o video abaixo:

<video width="640" height="480"
       src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/decoder_RNN.mp4"
       controls>
</video>

A classe decoder herda de nn.Module, que é uma classe base para todos os módulos de rede neural no PyTorch.
O construtor (método __init__) inicializa os parâmetros e camadas do decodificador.
- `output_dim` é o número de valores de saída possíveis (comprimento do vocabulário alvo).
- `emb_dim` é a dimensionalidade da camada de incorporação.
- `hid_dim` é a dimensionalidade do estado oculto no LSTM.
- `n_layers` é o número de camadas no LSTM.
- `dropout` é a probabilidade de abandono.

O decodificador contém as seguintes camadas:
- `embedding`: Uma camada de embedding que mapeia os valores de saída para vetores densos de tamanho emb_dim.
- `lstm`: Uma camada LSTM que pega a entrada incorporada e produz estados ocultos de tamanho hid_dim.
- `fc_out`: Uma camada linear que mapeia a saída LSTM para a dimensão de saída output_dim.
- `softmax`: Uma função de ativação log-softmax aplicada à saída para obter uma distribuição de probabilidade sobre os valores de saída.
- `dropout`: Uma camada de dropout que aplica dropout à entrada incorporada.

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim = 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        #input = [batch size]

        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)
        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        prediction_logit = self.fc_out(output.squeeze(0))
        prediction = self.softmax(prediction_logit)
        #prediction = [batch size, output dim]


        return prediction, hidden, cell       

Podemos criar uma instância de decodificador. A dimensão de saída é definida como o comprimento do vocabulário de destino.

In [17]:
output_dim = 6
emb_dim=10
hid_dim = 8
n_layers=1
dropout=0.5
decoder_t = Decoder(output_dim, emb_dim, hid_dim, n_layers, dropout).to(device)

Agora que temos instâncias de encoder e decoder, podemos conectá-los (a caixa vermelha no diagrama abaixo). Primeiro, vamos ver como você pode passar o Hidden e o Cell (a célula rosa dentro da caixa vermelha) do encoder (o contêiner de caixas verdes) para o decoder (o contêiner de caixas laranja). Olhando para o diagrama, você pode ver que o decoder também recebe uma entrada que é a palavra anterior que ele previu. Para a primeira célula do decoder, essa entrada é o token `<bos>`. Cada célula do decoder emite uma previsão e atualiza a célula e o estado para passar para a próxima célula do decoder. A previsão é uma distribuição de probabilidade sobre possíveis tokens de destino (comprimento do vocabulário de destino).
![connection](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/ED_connection.JPG)


In [18]:
input_t = torch.tensor([0]).to(device) #<bos>
input_t.shape
prediction, hidden, cell = decoder_t(input_t, hidden_t , cell_t)
print("Prediction:", prediction, '\n\nHidden:',hidden,'\n\nCell:', cell)

Prediction: tensor([[-2.1463, -1.4403, -1.8858, -1.6018, -1.9691, -1.8747]],
       grad_fn=<LogSoftmaxBackward0>) 

Hidden: tensor([[[ 0.1940,  0.2322, -0.1090, -0.2850, -0.1056,  0.2018,  0.1688,
           0.1350]]], grad_fn=<StackBackward0>) 

Cell: tensor([[[ 0.4253,  0.3850, -0.2375, -0.7614, -0.1726,  0.2291,  0.3454,
           0.4532]]], grad_fn=<StackBackward0>)


### Conectar Encoder-Decoder

Vimos como criar módulos codificadores e decodificadores e como passar entrada para eles. Agora precisamos criar a conexão para que o modelo possa processar pares (`src`,`trg`) e gerar a tradução. Suponha que `trg` seja tensor ([[0],[2],[3],[5],[1]]) que é igual à sequência 0,2,3,5,1 na qual cada número representa um token no vocabulário alvo. Por exemplo, 0:`<bos>`,2:"this", 3:"is",5:"beautiful", 1:`<eos>`.

In [19]:
#trg = [trg len, batch size]
#teacher_forcing_ratio is probability to use teacher forcing
#e.g. if teacher_forcing_ratio is 0.75 you use ground-truth inputs 75% of the time
teacher_forcing_ratio = 0.5
trg = torch.tensor([[0],[2],[3],[5],[1]]).to(device)

In [20]:
batch_size = trg.shape[1]
trg_len = trg.shape[0]
trg_vocab_size = decoder_t.output_dim

#tensor to store decoder outputs
outputs_t = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

In [21]:
#send to device

hidden_t = hidden_t.to(device)
cell_t = cell_t.to(device)

In [22]:
#first input to the decoder is the <bos> tokens
input = trg[0,:]

In [23]:
for t in range(1, trg_len):

    #you loop through the trg len and generate tokens
    #decoder receives previous generated token, cell and hidden
    # decoder outputs it prediction(probablity distribution for the next token) and updates hidden and cell
    output_t, hidden_t, cell_t = decoder_t(input, hidden_t, cell_t)

    #place predictions in a tensor holding predictions for each token
    outputs_t[t] = output_t

    #decide if you are going to use teacher forcing or not
    teacher_force = random.random() < teacher_forcing_ratio

    #get the highest predicted token from your predictions
    top1 = output_t.argmax(1)


    #if teacher forcing, use actual next token as next input
    #if not, use predicted token
    #input = trg[t] if teacher_force else top1
    input = trg[t] if teacher_force else top1

print(outputs_t,outputs_t.shape )

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-2.1373, -1.4752, -1.8961, -1.5600, -1.9611, -1.8819]],

        [[-2.1009, -1.4650, -1.8953, -1.5649, -2.0019, -1.8831]],

        [[-2.0453, -1.5394, -1.8731, -1.5384, -2.0219, -1.8619]],

        [[-1.9978, -1.6298, -1.7415, -1.6078, -1.9532, -1.8906]]],
       grad_fn=<CopySlices>) torch.Size([5, 1, 6])


O tamanho do tensor de saída é (trg_len, batch_size, trg_vocab_size). Isso ocorre porque para cada token `trg` (comprimento de `trg`) o modelo produz uma distribuição de probabilidade sobre todos os tokens possíveis (comprimento do vocabulário trg). Portanto, para gerar os tokens previstos ou a tradução da frase `src`, você precisa obter a probabilidade máxima para cada token:

In [24]:
# Note that you need to get the argmax from the second dimension as **outputs** is an array of **output** tensors
pred_tokens = outputs_t.argmax(2)
print(pred_tokens)

tensor([[0],
        [1],
        [1],
        [3],
        [3]])


Não é surpresa que a tradução não esteja correta (trg = tensor([[0],[2],[3],[5],[1]]), pois o modelo ainda não passou por nenhum treinamento.

### Implementação do modelo sequência-a-sequência (seq2seq) no PyTorch

Vamos conectar os componentes do codificador e do decodificador para criar o modelo seq2seq.

Você define a classe seq2seq que herda de nn.Module, que é a classe base para todos os módulos de rede neural no PyTorch.
As entradas são:
- `encoder` e `decoder` são instâncias das redes de codificador e decodificador que você já definiu.
- `device` especifica o dispositivo (por exemplo, CPU ou GPU) no qual os cálculos serão realizados.
- `trg_vocab` representa o vocabulário do idioma de destino. É usado para determinar o tamanho do vocabulário de saída.

O método **forward** define o passe para frente do modelo seq2seq. Ele recebe três argumentos: `src`, `trg` e `teacher_forcing_ratio`.:

- `src` representa as sequências de origem e `trg` representa as sequências de destino.
- `teacher_forcing_ratio` é uma probabilidade que determina se o forçamento do professor será usado apenas durante o treinamento. O forçamento do professor é uma técnica em que a sequência de destino verdadeira é alimentada como entrada para o decodificador em cada passo de tempo, em vez de usar a saída prevista do passo de tempo anterior.

O método **forward** inicializa algumas variáveis ​​necessárias para o passe para frente, como `batch_size`, `trg_len` e `trg_vocab_size`. Ele também cria um tensor vazio chamado `outputs` para armazenar as saídas do decodificador para cada passo de tempo.

Os estados `hidden` e `cell` do codificador são obtidos chamando o método encoder (src). Esses estados são então usados ​​como estados iniciais para o decodificador.

A entrada para o decodificador no primeiro passo de tempo é o token <bos> das sequências de destino.

O decodificador é iterado para cada passo de tempo nas sequências de destino (`for t in range(1, trg_len)`). A entrada, junto com os estados hidden e cell anteriores, é passada para o decodificador e produz um tensor de saída. O tensor `output` é armazenado no tensor `outputs`.

Em cada passo de tempo, há uma decisão tomada sobre usar força do professor ou não com base na probabilidade teacher_forcing_ratio. Se a força do professor for usada, o próximo token verdadeiro das sequências de destino (`trg[t]`) é usado como entrada para o próximo passo de tempo. Caso contrário, o token previsto do passo de tempo anterior (`top1 = output.argmax(1)`) é usado.

Finalmente, o tensor `outputs` contendo as saídas previstas para cada passo de tempo é retornado.

In [87]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, trg_vocab):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.trg_vocab = trg_vocab

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"


    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 you use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        hidden = hidden.to(device)
        cell = cell.to(device)

        # First input to the decoder is the <bos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if you are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from your predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            #input = trg[t] if teacher_force else top1
            input = trg[t] if teacher_force else top1


        return outputs

### Modelo de treinamento no PyTorch
Agora que o modelo está definido, vamos construir uma função de treinamento, o modelo seq2seq. Seus componentes:

1. `train(model, iterator, optimizer, criteria, clip)` recebe cinco argumentos:

- `model` é o modelo que será treinado.
- `iterator` é um objeto iterável que fornece os dados de treinamento em lotes.
- `optimizer` é o algoritmo de otimização usado para atualizar os parâmetros do modelo.
- `criterion` é a função de perda que mede o desempenho do modelo.
- `clip` é um valor usado para recortar os gradientes para evitar que eles se tornem muito grandes durante a retropropagação.

2. A função começa definindo o modelo para o modo de treinamento com `model.train()`. Isso é necessário para habilitar certas camadas (por exemplo, dropout) que se comportam de forma diferente durante o treinamento e a avaliação.

3. Inicializa uma variável `epoch_loss` para manter o controle da perda acumulada durante a época.

4. A função itera sobre os dados de treinamento fornecidos pelo `iterator`. Cada iteração recupera um lote de sequências de entrada (`src`) e sequências de destino (`trg`).

5. As sequências de entrada (`src`) e sequências de destino (`trg`) são movidas para o dispositivo apropriado (por exemplo, GPU) usando `src = src.to(device)` e `trg = trg.to(device)`.

6. Os gradientes dos parâmetros do modelo são limpos usando `optimizer.zero_grad()` para preparar o novo lote.

7. O modelo é então chamado com `output = model(src, trg)` para obter as previsões do modelo para as sequências de destino.

8. O tensor `output` tem dimensões `[trg len, batch size, output dim]`. Para calcular a perda, o tensor é remodelado para `[trg len - 1, batch size, output dim]` para remover o token `<bos>` inicial, que não é usado para calcular a perda.

9. As sequências de destino (`trg`) também são remodeladas para `[trg len - 1]` removendo o token `<bos>` inicial e tornando-o um tensor contíguo. Isso corresponde ao formato do tensor `output` remodelado.

10. A perda entre os tensores `output` e `trg` remodelados é calculada usando o `criterion` especificado.

11. Os gradientes da perda com relação aos parâmetros do modelo são computados usando `loss.backward()`.

12. Os gradientes são então recortados para um valor máximo especificado por `clip` usando `torch.nn.utils.clip_grad_norm_(model.parameters(), clip)`. Isso evita que os gradientes se tornem muito grandes, o que pode causar problemas durante a otimização.

13. O método `step()` do otimizador é chamado para atualizar os parâmetros do modelo usando os gradientes computados.

14. A perda atual do lote (`loss.item()`) é adicionada à variável `epoch_loss`.

15. Após todos os lotes terem sido processados, a função retorna a perda média por lote para toda a época, calculada como `epoch_loss / len(list(iterator))`.

In [110]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    # Wrap iterator with tqdm for progress logging
    #train_iterator = tqdm(iterator, desc="Training", leave=False)

    for i, (src,trg) in enumerate(iterator):

        src = src.to(device)
        trg = trg.to(device)
        optimizer.zero_grad()

        output = model(src, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)

        trg = trg[1:].contiguous().view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        # Update tqdm progress bar with the current loss
        #train_iterator.set_postfix(loss=loss.item())

        epoch_loss += loss.item()


    return epoch_loss / len(list(iterator))

### Avaliando modelo no PyTorch
Você também precisa definir uma função para avaliar o modelo. Vamos analisar o código e entender seus componentes:

1. `evaluate(model, iterator, criteria)` recebe três argumentos:
- `model` é o modelo de rede neural que será avaliado.
- `iterator` é um objeto iterável que fornece os dados de avaliação em lotes.
- `criterion` é a função de perda que mede o desempenho do modelo.
* Observe que a função de avaliação não realiza nenhuma otimização no modelo.

2. A função começa definindo o modelo para o modo de avaliação com `model.eval()`.

3. Ela inicializa uma variável `epoch_loss` para manter o controle da perda acumulada durante a avaliação.

4. A função entra em um bloco `with torch.no_grad()`, que garante que nenhum gradiente seja computado durante a avaliação. Isso economiza memória e acelera o processo de avaliação, pois os gradientes não são necessários para atualizações de parâmetros.

5. A função itera sobre os dados de avaliação fornecidos pelo `iterator`. Cada iteração recupera um lote de sequências de entrada (`src`) e sequências de destino (`trg`).

6. As sequências de entrada (`src`) e sequências de destino (`trg`) são movidas para o dispositivo apropriado (por exemplo, GPU) usando `src = src.to(device)` e `trg = trg.to(device)`.

7. O modelo é então chamado com `output = model(src, trg, 0)` para obter as previsões do modelo para as sequências alvo. O terceiro argumento `0` é passado para indicar que o forçamento do professor é desativado durante a avaliação. Durante a avaliação, o forçamento do professor é normalmente desativado para avaliar a capacidade do modelo de gerar sequências com base em suas próprias previsões.

8. O tensor `output` tem dimensões `[trg len, batch size, output dim]`. Para calcular a perda, o tensor é remodelado para `[trg len - 1, batch size, output dim]` para remover o token inicial `<bos>` (início da sequência), que não é usado para calcular a perda.

9. As sequências alvo (`trg`) também são remodeladas para `[trg len - 1]` removendo o token `<bos>` inicial e tornando-o um tensor contíguo. Isso corresponde ao formato do tensor `output` remodelado.

10. A perda entre os tensores `output` e `trg` remodelados é calculada usando o `criterion` especificado.

11. A perda do lote atual (`loss.item()`) é adicionada à variável `epoch_loss`.

12. Após todos os lotes terem sido processados, a função retorna a perda média por lote para toda a avaliação, calculada como `epoch_loss / len(list(iterator))`.

In [95]:
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    # Wrap iterator with tqdm for progress logging
    valid_iterator = tqdm(iterator, desc="Training", leave=False)

    with torch.no_grad():

        for i, (src,trg) in enumerate(iterator):

            src = src.to(device)
            trg = trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)

            trg = trg[1:].contiguous().view(-1)


            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            # Update tqdm progress bar with the current loss
            valid_iterator.set_postfix(loss=loss.item())

            epoch_loss += loss.item()

    return epoch_loss / len(list(iterator))

### Processamento dos dados

Buscar um conjunto de dados de tradução de idioma chamado Multi30k, o agrupará (tokenização, numericização e adição de BOS/EOS e preenchimento) e criará lotes iteráveis de tensores src e trg.

Isso aproveita o collate_fn predefinido para curar e preparar lotes de forma eficiente para treinar o modelo do transformador. O objetivo principal é se aprofundar nas complexidades dos componentes do codificador e decodificador RNN.

Foi criado um arquivo "Multi30K_de_en_dataloader.py" que contém todos os processos de transformação em dados. Aqui, você só baixa o arquivo:

In [96]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Multi30K_de_en_dataloader.py'

import requests
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Multi30K_de_en_dataloader.py'
filename = 'Multi30K_de_en_dataloader.py'

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded: {filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded: Multi30K_de_en_dataloader.py


In [97]:
#!python -m spacy download de_core_news_sm

In [98]:
import spacy

try:
    nlp = spacy.load("de_core_news_sm")
    print("Modelo carregado com sucesso!")
except OSError as e:
    print(f"Erro ao carregar o modelo: {e}")

Modelo carregado com sucesso!


In [114]:
%run Multi30K_de_en_dataloader.py

chamar a função `get_translation_dataloaders(batch_size = N,flip=True)` com um tamanho de lote arbitrário `N` e definir flip como True para que o codificador LSTM receba a sequência de entrada na ordem inversa. Isso pode ajudar no treinamento.


In [100]:
train_dataloader, valid_dataloader = get_translation_dataloaders(batch_size = 4)#,flip=True)

In [101]:
# check the src and trg tensors
src, trg = next(iter(train_dataloader))
src,trg

(tensor([[    2,     2,     2,     2],
         [    3,  5510,  5510, 12642],
         [    1,     3,     3,     8],
         [    1,     1,     1,  1701],
         [    1,     1,     1,     3]]),
 tensor([[   2,    2,    2,    2],
         [   3, 6650,  216,    6],
         [   1, 4623,  110, 3398],
         [   1,  259, 3913,  202],
         [   1,  172, 1650,  109],
         [   1, 9953, 3823,   37],
         [   1,  115,   71,    3],
         [   1,  692, 2808,    1],
         [   1, 3428, 2187,    1],
         [   1,    5,    5,    1],
         [   1,    3,    3,    1]]))

Podemos obter as strings em inglês e alemão usando as funções `index_to_eng` e `index_to_german` fornecidas no arquivo .py:

In [102]:
data_itr = iter(train_dataloader)
# moving forward in the dataset to reach sequences of longer length for illustration purpose. (Remember the dataset is sorted on sequence len for optimal padding)
for n in range(1000):
    german, english= next(data_itr)

for n in range(3):
    german, english=next(data_itr)
    german=german.T
    english=english.T
    print("________________")
    print("german")
    for g in german:
        print(index_to_german(g))
    print("________________")
    print("english")
    for e in english:
        print(index_to_eng(e))

________________
german
<bos> Personen mit schwarzen Hüten in der Innenstadt . <eos>
<bos> Eine Gruppe Menschen protestiert in einer Stadt . <eos>
<bos> Eine Gruppe teilt ihre politischen Ansichten mit . <eos>
<bos> Mehrere Personen sitzen an einem felsigen Strand . <eos>
________________
english
<bos> People in black hats gathered together downtown . <eos> <pad> <pad> <pad>
<bos> A group of people protesting in a city . <eos> <pad> <pad>
<bos> A group is letting their political opinion be known . <eos> <pad>
<bos> A group of people are sitting on a rocky beach . <eos>
________________
german
<bos> Zwei sitzende Personen mit Hüten und Sonnenbrillen . <eos>
<bos> Ein kleiner Junge mit Hut beim Angeln . <eos>
<bos> Diese zwei Frauen haben Spaß im Giorgio's . <eos>
<bos> Zwei kleine Kinder schlafen auf dem Sofa . <eos>
________________
english
<bos> Two people sitting in hats and shades . <eos> <pad> <pad> <pad>
<bos> A young boy in a hat is fishing by himself . <eos>
<bos> These two wome

* Nota: Ao trabalhar com tensores PyTorch que representam dados, é importante entender as convenções em torno da representação de sequências. Na maioria dos casos, as linhas (a primeira dimensão) em um tensor PyTorch representam amostras individuais, enquanto as colunas (a segunda dimensão) representam recursos ou etapas de tempo no caso de sequências. Ao lidar com sequências no PyTorch, é comum usar funções como `pad_sequence` para garantir que todas as sequências tenham o mesmo comprimento. Surpreendentemente, a operação de preenchimento é aplicada ao longo da segunda dimensão (colunas), embora as sequências sejam tipicamente representadas na primeira dimensão (linhas). Isso pode ser confuso no início devido à maneira como os lotes de sequências são representados. Em muitas tarefas relacionadas a sequências no PyTorch, especialmente ao trabalhar com modelos recorrentes como RNNs, LSTMs e GRUs, lotes de sequências são geralmente representados com a forma [sequence_length, batch_size, feature_size], onde `sequence_length` se refere ao comprimento da sequência mais longa dentro do lote (aqui é equivalente a `src_len` ou `trg_len`). Se você verificar o tensor src acima, poderá ver que a primeira palavra de todas as frases está na primeira linha, a segunda palavra de todas as frases está na segunda linha, etc. É por isso que a primeira dimensão é o comprimento da sequência.

* Quando você usa `pad_sequence`, ele adiciona preenchimento às sequências em um lote para que todas tenham o mesmo comprimento, correspondendo ao comprimento da sequência mais longa. Como as sequências são representadas na primeira dimensão, o preenchimento é aplicado ao longo dessa dimensão. Como resultado, o tensor de saída de `pad_sequence` terá o formato [sequence_length, batch_size]. (Verifique a saída para `src` e `trg` da célula acima.) Essa convenção é comumente usada porque modelos como LSTMs esperam que os dados estejam neste formato. No entanto, se você estiver acostumado a trabalhar com dados tabulares mais tradicionais no PyTorch, isso pode causar confusão inicialmente. É importante estar ciente dessa convenção para evitar erros potenciais e entender como preparar e formatar adequadamente os dados de sequência para seus modelos.

### Treinar o modelo
#### Inicializações
Este código define a semente aleatória para várias bibliotecas e módulos. Isso é feito para tornar os resultados reproduzíveis:

In [103]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Treinamento
Agora, vamos definir a instância do modelo:

- `enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)`: Esta linha cria uma instância da classe `Encoder`, que representa o componente codificador do modelo Seq2Seq. A classe `Encoder` pega a dimensão de entrada, dimensão de incorporação, dimensão oculta, número de camadas e probabilidade de abandono como argumentos.

- `dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)`: Esta linha cria uma instância da classe `Decoder`, que representa o componente decodificador do modelo Seq2Seq. A classe `Decoder` usa a dimensão de saída, dimensão de incorporação, dimensão oculta, número de camadas e probabilidade de abandono como argumentos.

- `model = Seq2Seq(enc, dec, device,trg_vocab = vocab_transform['en']).to(device)`: Esta linha cria uma instância da classe `Seq2Seq`, que representa todo o modelo Seq2Seq. A classe `Seq2Seq` usa o codificador, o decodificador e o dispositivo (por exemplo, CPU ou GPU) como argumentos. Ela combina o codificador e o decodificador para formar a arquitetura Seq2Seq completa.

In [104]:
INPUT_DIM = len(vocab_transform['de'])
OUTPUT_DIM = len(vocab_transform['en'])
ENC_EMB_DIM = 128 #256
DEC_EMB_DIM = 128 #256
HID_DIM = 256 #512
N_LAYERS = 1 #2
ENC_DROPOUT = 0.3 #0.5
DEC_DROPOUT = 0.3 #0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device,trg_vocab = vocab_transform['en']).to(device)

`def init_weights(m)`define uma função chamada `init_weights` que recebe um módulo `m` como entrada. O propósito desta função é inicializar os pesos do módulo da rede neural.

A próxima linha `for name, param in m.named_parameters():` inicia um loop que itera sobre os parâmetros nomeados do módulo `m`. Cada parâmetro é acessado como `param` e seu nome correspondente é acessado como `name`.

`nn.init.uniform_(param.data, -0.08, 0.08)`inicializa os dados do parâmetro com valores extraídos de uma distribuição uniforme entre `-0.08` e `0.08`. A função `nn.init.uniform_` é fornecida pela biblioteca PyTorch e é usada para inicializar os pesos dos parâmetros da rede neural.

Finalmente, `model.apply(init_weights)` aplica a função `init_weights` à instância `model`. Isso garante que os pesos de todos os parâmetros no modelo sejam inicializados usando a distribuição uniforme especificada.

In [105]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19214, 128)
    (lstm): LSTM(128, 256, dropout=0.3)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10837, 128)
    (lstm): LSTM(128, 256, dropout=0.3)
    (fc_out): Linear(in_features=256, out_features=10837, bias=True)
    (softmax): LogSoftmax(dim=1)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (trg_vocab): Vocab()
)

Este código define uma função `count_parameters` que conta o número de parâmetros treináveis ​​em um dado modelo. Ele então imprime a contagem de parâmetros treináveis ​​em uma string formatada.

In [106]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,422,165 trainable parameters


A célula a seguir configura o otimizador e a função de perda para treinar o modelo.

1. `optimizer = optim.Adam(model.parameters())`: Esta linha cria uma instância do otimizador Adam e passa os parâmetros do modelo (`model.parameters()`) como os parâmetros a serem otimizados. O otimizador Adam é um algoritmo de otimização popular comumente usado para treinar redes neurais profundas. Ele ajusta os parâmetros do modelo com base nos gradientes computados durante a retropropagação para minimizar a função de perda.

2. `PAD_IDX = vocab_transform['en'].get_stoi()['<pad>']`: ​​Esta linha recupera o índice do token `<pad>` no vocabulário de destino.

3. `criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)`: Esta linha cria uma instância do critério CrossEntropyLoss. CrossEntropyLoss é uma função de perda comumente usada para tarefas de classificação multiclasse. Nesse caso, ela é usada para treinar o modelo para prever a próxima palavra na sequência traduzida. O parâmetro `ignore_index` é definido como `PAD_IDX`, o que indica que a perda deve ser ignorada para quaisquer previsões em que o alvo seja o token de preenchimento. Isso é útil para excluir tokens de preenchimento de contribuir para a perda durante o treinamento.

In [107]:
optimizer = optim.Adam(model.parameters())

PAD_IDX = vocab_transform['en'].get_stoi()['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

A função auxiliar a seguir fornece uma maneira conveniente de calcular o tempo decorrido em minutos e segundos, dados os horários de início e fim. Ela será usada para medir o tempo gasto para cada época durante o treinamento ou quaisquer outros cálculos relacionados ao tempo.

In [108]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Esteja ciente de que treinar o modelo usando CPUs pode ser um processo demorado. Se você não tiver acesso a GPUs, pode pular para "carregando o modelo salvo" e prosseguir com o carregamento do modelo pré-treinado usando o código fornecido.

Vamos começar as épocas de treinamento:

In [112]:
N_EPOCHS = 3  # Number of epochs
CLIP = 1

best_valid_loss = float('inf')
train_losses = []
valid_losses = []
train_PPLs = []
valid_PPLs = []

# Initialize tqdm for progress logging
progress_bar = tqdm(range(N_EPOCHS), desc="Training", unit="epoch")

for epoch in progress_bar:
    start_time = time.time()

    # Call train function
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    train_ppl = math.exp(train_loss)

    # Call evaluate function
    valid_loss = evaluate(model, valid_dataloader, criterion)
    valid_ppl = math.exp(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'RNN-TR-model.pt')

    train_losses.append(train_loss)
    train_PPLs.append(train_ppl)
    valid_losses.append(valid_loss)
    valid_PPLs.append(valid_ppl)

    # Update the description of the tqdm bar dynamically
    progress_bar.set_description(
        f"Epoch {epoch+1}/{N_EPOCHS} | "
        f"Train Loss: {train_loss:.3f}, Train PPL: {train_ppl:.3f} | "
        f"Val Loss: {valid_loss:.3f}, Val PPL: {valid_ppl:.3f}"
    )


Training:   0%|                                                                               | 0/3 [13:34<?, ?epoch/s]


KeyboardInterrupt: 

Visualizar as perdas do modelo de treinamento e validação ao longo dos períodos de treinamento:

In [ ]:
import matplotlib.pyplot as plt



# Create a list of epoch numbers
epochs = [epoch+1 for epoch in range(N_EPOCHS)]

# Create the figure and axes
fig, ax1 = plt.subplots(figsize=(10, 6))
ax2 = ax1.twinx()

# Plotting the training and validation loss
ax1.plot(epochs, train_losses, label='Train Loss', color='blue')
ax1.plot(epochs, valid_losses, label='Validation Loss', color='orange')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss/PPL')

# Plotting the training and validation perplexity
ax2.plot(epochs, train_PPLs, label='Train PPL', color='green')
ax2.plot(epochs, valid_PPLs, label='Validation PPL', color='red')
ax2.set_ylabel('Perplexity')

# Adjust the y-axis scaling for PPL plot
ax2.set_ylim(bottom=min(min(train_PPLs), min(valid_PPLs)) - 10, top=max(max(train_PPLs), max(valid_PPLs)) + 10)

# Set the legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2
ax1.legend(lines, labels, loc='upper right')


# Show the plot
plt.show()

### Carregando o modelo salvo
Se você quiser pular o treinamento e carregar o modelo pré-treinado

In [115]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/RNN-TR-model.pt'
filename = 'RNN-TR-model.pt'

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded: {filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded: RNN-TR-model.pt


In [116]:
model.load_state_dict(torch.load('RNN-TR-model.pt',
                                 map_location=torch.device('cpu')))

<All keys matched successfully>

### Inferência de modelo

Em seguida, crie uma função geradora que gere traduções para frases de fonte de entrada:

In [117]:
import torch.nn.functional as F

def generate_translation(model, src_sentence, src_vocab, trg_vocab, max_len=50):
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        src_tensor = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1).to(device)

        # Pass the source tensor through the encoder
        hidden, cell = model.encoder(src_tensor)

        # Create a tensor to store the generated translation
        # get_stoi() maps tokens to indices
        trg_indexes = [trg_vocab.get_stoi()['<bos>']]  # Start with <bos> token

        # Convert the initial token to a PyTorch tensor
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1)  # Add batch dimension

        # Move the tensor to the same device as the model
        trg_tensor = trg_tensor.to(model.device)


        # Generate the translation
        for _ in range(max_len):

            # Pass the target tensor and the previous hidden and cell states through the decoder
            output, hidden, cell = model.decoder(trg_tensor[-1], hidden, cell)

            # Get the predicted next token
            pred_token = output.argmax(1)[-1].item()

            # Append the predicted token to the translation
            trg_indexes.append(pred_token)


            # If the predicted token is the <eos> token, stop generating
            if pred_token == trg_vocab.get_stoi()['<eos>']:
                break

            # Convert the predicted token to a PyTorch tensor
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1)  # Add batch dimension

            # Move the tensor to the same device as the model
            trg_tensor = trg_tensor.to(model.device)

        # Convert the generated tokens to text
        # get_itos() maps indices to tokens
        trg_tokens = [trg_vocab.get_itos()[i] for i in trg_indexes]

        # Remove the <sos> and <eos> from the translation
        if trg_tokens[0] == '<bos>':
            trg_tokens = trg_tokens[1:]
        if trg_tokens[-1] == '<eos>':
            trg_tokens = trg_tokens[:-1]

        # Return the translation list as a string

        translation = " ".join(trg_tokens)

        return translation

Verificar a saída do modelo para uma frase de exemplo:

In [119]:
# Actual translation: Asian man sweeping the walkway.
src_sentence = 'Ein asiatischer Mann kehrt den Gehweg.'


generated_translation = generate_translation(model, src_sentence=src_sentence, src_vocab=vocab_transform['de'], trg_vocab=vocab_transform['en'], max_len=12)
#generated_translation = " ".join(generated_translation_list).replace("<bos>", "").replace("<eos>", "")
print(generated_translation)

An Asian man is on the sidewalk .


### Métrica de pontuação BLEU para avaliação
Embora a peplexity sirva como uma métrica geral para avaliar o desempenho do modelo de linguagem na previsão do próximo token correto, a pontuação BLEU é útil para avaliar a qualidade da tradução final gerada.
Validar os resultados usando a pontuação BLEU é útil quando há mais de uma tradução válida para uma frase, pois você pode incluir muitas versões de tradução na lista de referência e comparar a tradução gerada com diferentes versões de traduções.

A pontuação BLEU (Bilingual Evaluation Understudy) é uma métrica comumente usada para avaliar a qualidade de traduções geradas por máquina, comparando-as a uma ou mais traduções de referência. Ela mede a similaridade entre a tradução gerada e as traduções de referência com base na correspondência de n-gramas.

A pontuação BLEU é calculada usando as seguintes fórmulas:

1. **Precisão**:
- A precisão mede a proporção de n-gramas na tradução gerada que aparecem nas traduções de referência.
- A precisão é calculada para cada ordem de n-gramas (1 a N) e então combinada usando uma média geométrica.
- A precisão para uma ordem de n-gramas específica é calculada como:
   $$\text{Precision}_n(t) = \frac{\text{CountClip}_n(t)}{\text{Count}_n(t)}$$
  onde:
- $\text{CountClip}_n(t)$ é a contagem de n-gramas na tradução gerada que aparecem em qualquer tradução de referência, cortada pela contagem máxima desse n-grama em qualquer tradução de referência única.
- $\text{Count}_n(t)$ é a contagem de n-gramas na tradução gerada.

2. . **Penalidade de brevidade**:
- A penalidade de brevidade é responsável pelo fato de que traduções mais curtas tendem a ter pontuações de precisão mais altas.
- Ela incentiva traduções que são mais próximas em comprimento das traduções de referência.
- A penalidade de brevidade é calculada como:

$$\text{BP} = \begin{cases} 1 & \text{if } c > r \\\\ e^{(1 - \frac{r}{c})} & \text{if } c \leq r \end{cases}$$

onde:
- $c$ é o comprimento total da tradução gerada.
- $r$ é o comprimento total das traduções de referência.

3. **Pontuação BLEU**:
- A pontuação BLEU é a média geométrica das precisões, ponderada pela penalidade de brevidade.
- É calculada como:

$$\text{BLEU} = \text{BP} \cdot \exp(\sum_{n=1}^{N}w_n \log(\text{Precisão}_n(t)))$$

onde:
- $N$ é a ordem máxima de n-gramas.
- $w_n$ é o peso atribuído à precisão na ordem de n-gramas $n$, comumente definido como $\frac{1}{N}$ para pesos iguais.

In [120]:
def calculate_bleu_score(generated_translation, reference_translations):
    # Convert the generated translations and reference translations into the expected format for sentence_bleu
    references = [reference.split() for reference in reference_translations]
    hypothesis = generated_translation.split()

    # Calculate the BLEU score
    bleu_score = sentence_bleu(references, hypothesis)

    return bleu_score

In [121]:
reference_translations = [
    "Asian man sweeping the walkway .",
    "An asian man sweeping the walkway .",
    "An Asian man sweeps the sidewalk .",
    "An Asian man is sweeping the sidewalk .",
    "An asian man is sweeping the walkway .",
    "Asian man sweeping the sidewalk ."
]

bleu_score = calculate_bleu_score(generated_translation, reference_translations)
print("BLEU Score:", bleu_score)

BLEU Score: 0.5


In [122]:
german_text = "Menschen gehen auf der Straße"

# The function should be defined to accept the text, the model, source and target vocabularies, and the device as parameters.
english_translation = generate_translation(
    model, 
    src_sentence=german_text, 
    src_vocab=vocab_transform['de'], 
    trg_vocab=vocab_transform['en'], 
    max_len=50
)

# Display the original and translated text
print(f"Original German text: {german_text}")
print(f"Translated English text: {english_translation}")

Original German text: Menschen gehen auf der Straße
Translated English text: People are walking on the street .


____
Esse material tem como referência o curso [Gen AI Foundational Models for NLP & Language Understanding](https://www.coursera.org/learn/gen-ai-foundational-models-for-nlp-and-language-understanding?specialization=generative-ai-engineering-with-llms)